### **PRE PROCESSAMENTO**

In [ ]:
import pandas as pd 
df = pd.DataFrame()

In [ ]:
def remove_outlier(df):
    Q1 = df.quatile(0.25)
    Q3 = df.quatile(0.75)
    IQ = Q3 - Q1
    LI = df < (Q1 - 1.5*IQ)
    LS = df > (Q3 + 1.5*IQ)

    df_final = df[~(LI | LS)]
    return df_final

In [ ]:
# Removendo Outliers de um dataFrame inteiro
df_normal = remove_outlier(df)

# Se quiser saber a diferença entre um DataFrame e outro:
    # Retorna apenas os indices das linhas que contem os outiliers
indice_dif = df.index.difference(df_normal.index)

In [ ]:
for i in range(indice_dif, 1):
    df.loc[df_dif[i], 'coluna_que_quero'] = media_de_algum_valor

### **VERIFICANDO QUANTIDADE DE CLUSTERs**

In [ ]:
# Calculando a quantidade de Clusters com base na Soma dos Quadrados (Cotovelo)
def qtd_cluster(df):
    wcss = []
    for i in range(1, 11):  # Testando de 1 a 10 clusters
        kmeans = KMeans(n_clusters=i, random_state=42)  # Adicionando random_state para resultados consistentes
        kmeans.fit(df)  
        wcss.append(kmeans.inertia_)

    # Criando o gráfico do cotovelo
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
    plt.title('Método do Cotovelo')
    plt.xlabel('Número de Clusters')
    plt.ylabel('WCSS (Within-Cluster Sum of Squares)')
    plt.show()

    return wcss



# Função para calcular a quantidade ideal de Clusters para minha base
# Soma dos Quadrados é o Vetor anterior 
def numero_otimo_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss) - 1]

    distancia = []
    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerador = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominador = math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
        distancia.append(numerador / denominador)
    
    return distancia.index(max(distancia)) + 2

In [ ]:
def quantidade_exata(wcss):
    x1, y1 = 1, wcss[0]
    x2, y2 = len(wcss), wcss[-1]
    
    print(f"x1, y1: {x1}, {y1}")
    print(f"x2, y2: {x2}, {y2}")
    
    distancia = []
    for i in range(len(wcss)):
        x0 = i + 1
        y0 = wcss[i]
        numerador = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominador = math.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
        
        print(f"x0, y0: {x0}, {y0}")
        print(f"numerador: {numerador}, denominador: {denominador}")
        
        if denominador == 0:
            print("Denominador é zero, ajuste necessário!")
            continue
        
        distancia.append(numerador / denominador)
    
    print(f"Distâncias calculadas: {distancia}")
    return distancia.index(max(distancia)) + 1

### **CRIANDO KLUSTER**

In [ ]:
kmenas= KMeans(n_clusters=5, max_iter=10000, init='random')
df_usuario['RecenciaCluster'] = kmenas.fit_predict(df_recencia)
df_usuario.head()

### **ORDENAÇÃO DO CLUSTER**

In [ ]:
# cluster_name = Nome do Cluster que vai para a tabela final
# Terget = Coluna que vou usar para calcular o valor me´dio que servirá de referencia para ordenalção dos dados
# DataFrame onde os dados serão salvos

def ordenar_cluster(cluster_name, target, df):
    agrupado_por_cluster = df.groupby(cluster_name)[target].mean().reset_index()
    agrupado_por_cluster_ordenado = agrupado_por_cluster.sort_values(by=target, ascending=True).reset_index(drop=True)
    agrupado_por_cluster_ordenado['index'] = agrupado_por_cluster_ordenado.index
    juntando_cluster = pd.merge(df, agrupado_por_cluster_ordenado[[cluster_name, 'index']], on = cluster_name)
    removendo_dados = juntando_cluster.drop([cluster_name], axis=1)
    df_final = removendo_dados.rename(columns={'index':cluster_name})
    return df_final
    
    

### **VISUALIZAÇÃO DOS CLUSTERS**

In [ ]:
# Cuidar a ordenação para ver se ela veio de forma correta
def plot_segmento(x, y, data):
    sns.set()  # Configura o estilo do gráfico
    plt.figure(figsize=(7, 5))  # Define o tamanho da figura
    sns.scatterplot(x=x, y=y, hue='Segmento', data=data, 
                    size='Segmento', sizes=(50, 150), 
                    size_order=['Premium', 'Master', 'Busines', 'Inativo'], 
                    palette='crest')  # Define a paleta de cores

    plt.show()  # Exibe o gráfico
    

## **DOCUMENTACAO**

class sklearn.cluster.KMeans(n_clusters=8, *, init='k-means++', n_init='auto', max_iter=300, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='lloyd')[source]
K-Means clustering.

Read more in the User Guide.

Parameters:
n_clustersint, default=8
The number of clusters to form as well as the number of centroids to generate.

For an example of how to choose an optimal value for n_clusters refer to Selecting the number of clusters with silhouette analysis on KMeans clustering.

init{‘k-means++’, ‘random’}, callable or array-like of shape (n_clusters, n_features), default=’k-means++’
Method for initialization:

‘k-means++’ : selects initial cluster centroids using sampling based on an empirical probability distribution of the points’ contribution to the overall inertia. This technique speeds up convergence. The algorithm implemented is “greedy k-means++”. It differs from the vanilla k-means++ by making several trials at each sampling step and choosing the best centroid among them.

‘random’: choose n_clusters observations (rows) at random from data for the initial centroids.

If an array is passed, it should be of shape (n_clusters, n_features) and gives the initial centers.

If a callable is passed, it should take arguments X, n_clusters and a random state and return an initialization.

For an example of how to use the different init strategy, see the example entitled A demo of K-Means clustering on the handwritten digits data.

n_init‘auto’ or int, default=’auto’
Number of times the k-means algorithm is run with different centroid seeds. The final results is the best output of n_init consecutive runs in terms of inertia. Several runs are recommended for sparse high-dimensional problems (see Clustering sparse data with k-means).

When n_init='auto', the number of runs depends on the value of init: 10 if using init='random' or init is a callable; 1 if using init='k-means++' or init is an array-like.

Added in version 1.2: Added ‘auto’ option for n_init.

Changed in version 1.4: Default value for n_init changed to 'auto'.

max_iterint, default=300
Maximum number of iterations of the k-means algorithm for a single run.

tolfloat, default=1e-4
Relative tolerance with regards to Frobenius norm of the difference in the cluster centers of two consecutive iterations to declare convergence.

verboseint, default=0
Verbosity mode.

random_stateint, RandomState instance or None, default=None
Determines random number generation for centroid initialization. Use an int to make the randomness deterministic. See Glossary.

copy_xbool, default=True
When pre-computing distances it is more numerically accurate to center the data first. If copy_x is True (default), then the original data is not modified. If False, the original data is modified, and put back before the function returns, but small numerical differences may be introduced by subtracting and then adding the data mean. Note that if the original data is not C-contiguous, a copy will be made even if copy_x is False. If the original data is sparse, but not in CSR format, a copy will be made even if copy_x is False.

algorithm{“lloyd”, “elkan”}, default=”lloyd”
K-means algorithm to use. The classical EM-style algorithm is "lloyd". The "elkan" variation can be more efficient on some datasets with well-defined clusters, by using the triangle inequality. However it’s more memory intensive due to the allocation of an extra array of shape (n_samples, n_clusters).

